<a href="https://colab.research.google.com/github/mark-rtb/simple_recommendation/blob/master/simple_recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [189]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"markrtb","key":"5e67f6a11a8606115b2e0f4a92c3d169"}'}

In [190]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json
!pip install -q kaggle
!pip install -q kaggle-cli
!kaggle datasets download -d snap/amazon-fine-food-reviews

kaggle.json
total 4
-rw------- 1 root root 63 Feb  9 17:34 kaggle.json
{"username":"markrtb","key":"5e67f6a11a8606115b2e0f4a92c3d169"}Downloading amazon-fine-food-reviews.zip to /content/amazon-fine-food-reviews
 96% 232M/242M [00:01<00:00, 154MB/s]
100% 242M/242M [00:01<00:00, 132MB/s]


In [0]:
import zipfile
with zipfile.ZipFile('amazon-fine-food-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall('amazon-fine-food-reviews')

In [192]:
import os
os.chdir('amazon-fine-food-reviews')
print(os.listdir())

['Reviews.csv', 'hashes.txt', 'database.sqlite']


In [0]:
import pandas as pd

In [194]:
import sqlite3
con=sqlite3.connect('database.sqlite')
df_data=pd.read_sql_query(""" SELECT * from Reviews""", con)[['UserId', 'ProductId', 'Score', 'Time']]
df_data.head()

,UserId,ProductId,Score,Time
0,A3SGXH7AUHU8GW,B001E4KFG0,5,1303862400
1,A1D87F6ZCVE5NK,B00813GRG4,1,1346976000
2,ABXLMWJIXXAIN,B000LQOCH0,4,1219017600
3,A395BORC6FGVXV,B000UA0QIQ,2,1307923200
4,A1UQRSCLF8GW1T,B006K2ZZ7K,5,1350777600


In [0]:
from fastai.collab import * 
model_dir="/tmp/model/"
path="/tmp/"

In [196]:
data = CollabDataBunch.from_df(df_data, user_name='UserId', item_name='ProductId', rating_name='Score')
data.show_batch()

UserId,ProductId,target
A2SZLNSI5KOQJT,B007M832YY,4.0
A3T29DKWJGSNPH,B002I9X4TU,5.0
AAA0TUKS5VBSA,B000E1HVR0,2.0
A34M2CRL6JDXBB,B008O2EHNC,5.0
A1BL9MWYKTIHA3,B000GAW23W,3.0


In [0]:
model_temp = collab_learner(data, n_factors=50, y_range=(int(df_data['Score'].min()), int(df_data['Score'].max())),  model_dir=model_dir, path=path)

In [198]:
model_temp.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,2.284398,2.225854,00:56
1,2.018283,2.036263,00:56
2,1.966967,1.982471,00:56
3,1.869651,1.927677,00:56
4,1.696820,1.918950,00:56


In [0]:

data_full = CollabDataBunch.from_df(df_data, valid_pct=0, user_name='UserId', item_name='ProductId', rating_name='Score')
full_model = collab_learner(data_full, n_factors=50, y_range=(int(df_data['Score'].min()), int(df_data['Score'].max())),  model_dir=model_dir, path=path)

In [200]:
full_model.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,2.147330,#na#,01:03
1,2.064570,#na#,01:03
2,1.939179,#na#,01:03
3,1.832900,#na#,01:04
4,1.740865,#na#,01:02


In [0]:
full_model.save('rec_model')

In [0]:
model_recomend_food = collab_learner(data_full, n_factors=50, y_range=(int(df_data['Score'].min()), int(df_data['Score'].max())),  model_dir=model_dir, path=path)

In [204]:
model_recomend_food.load('rec_model')

CollabLearner(data=TabularDataBunch;

Train: LabelList (568454 items)
x: CollabList
UserId A3SGXH7AUHU8GW; ProductId B001E4KFG0; ,UserId A1D87F6ZCVE5NK; ProductId B00813GRG4; ,UserId ABXLMWJIXXAIN; ProductId B000LQOCH0; ,UserId A395BORC6FGVXV; ProductId B000UA0QIQ; ,UserId A1UQRSCLF8GW1T; ProductId B006K2ZZ7K; 
y: FloatList
5.0,1.0,4.0,2.0,5.0
Path: .;

Valid: LabelList (0 items)
x: CollabList

y: FloatList

Path: .;

Test: None, model=EmbeddingDotBias(
  (u_weight): Embedding(256060, 50)
  (i_weight): Embedding(74259, 50)
  (u_bias): Embedding(256060, 1)
  (i_bias): Embedding(74259, 1)
), opt_func=functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.99)), loss_func=FlattenedLoss of MSELoss(), metrics=[], true_wd=True, bn_wd=True, wd=0.01, train_bn=True, path=PosixPath('/tmp'), model_dir='/tmp/model/', callback_fns=[functools.partial(<class 'fastai.basic_train.Recorder'>, add_time=True, silent=False)], callbacks=[], layer_groups=[Sequential(
  (0): Embedding(256060, 50)
  (

In [0]:
def food_recommendation(user_id, model_rec, base_data, number_food=3):

    user_w = model_rec.weight([user_id], is_item=False)

    list_food = list(base_data.loc[base_data['UserId']==user_id]['ProductId'])
    list_new_food = list(pd.unique(base_data.loc[~base_data['ProductId'].isin(list_food)]['ProductId']))
    foor_w = learn.weight(list_new_food, is_item=True)
    matrix_recomend = foor_w*user_w
    recomwnd_rating = torch.sum(matrix_recomend, dim=1)
    idx = (-np.array(recomwnd_rating)).argsort()[:number_food]
    for index in idx:
        print(list_new_food[index])




In [206]:
food_recommendation('A395BORC6FGVXV', model_recomend_food, df_data, number_food=3)

B005PXZ6JM
B000REI2X6
B001E5DXEU
